In [ ]:
path = '/content/gdrive/MyDrive/MultiProject'

In [ ]:
!pip install tensorflow
!pip install scikit-learn
!pip install pandas
!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=527cc41c9c9c44c7d2d5ca752fb010115b1bf2f7765534c5131a540b8379ea31
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https:/

In [ ]:
#필요한거임포트
from IPython.display import clear_output 

clear_output()

#%tensorflow_version 1.x
import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece


import pandas as pd
import numpy as np  
import re
import pickle
import dill #함수를 파일로 저장하기 위해
import logging # 로그출력문과 관련된건듯
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_addons as tfa #활성함수와 관련
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
import shutil

Instructions for updating:
non-resource variables are not supported in the long term


1.0.2
2.11.0
4.8.2
0.19.0


In [ ]:
!pip install keras-bert
!pip install keras-radam

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
import os

from keras_radam import RAdam

#경로잘연결돼있는지확인
#os.listdir('/root/project/bert')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Random seed 고정
#tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
df_train=pd.read_csv("/your_path/trainingdataset/감성대화말뭉치_Training.xlsx - Sheet1.csv", encoding = "utf-8", header = 0, index_col = 0)
df_test=pd.read_csv("/your_path/trainingdataset/감성대화말뭉치_Validation.xlsx - Sheet1.csv", encoding = "utf-8", header = 0, index_col = 0)

In [ ]:
df_train = df_train.drop_duplicates('문장')
df_train = df_train.reset_index(drop=True)
df_test = df_test.drop_duplicates('문장')
df_test = df_test.reset_index(drop=True)

#train 중복 29개행 제거됨.


#df_train['감정_대분류'].unique()
#df_test['감정_대분류'].unique()

In [ ]:
#분노 제거 - 가사 분류용 모델 만들기 위해서
#분노를 포함해서 분류했더니, 감성대화말뭉치 데이터에서 다루는 분노 감정에 알맞는 노래가 잘 없어서
#애매한 가사들이 다 분노로 분류되는 문제가 발생했기 때문임
angry = df_train[df_train['감정_대분류'].str.contains('분노')].index
nadf_train = df_train.drop(angry)
angry2 = df_test[df_test['감정_대분류'].str.contains('분노')].index
nadf_test = df_test.drop(angry2)

In [ ]:
emotion_map = {"불안":0,"분노":1, "상처":2,"슬픔":3,"당황":4,"기쁨":5}
df_train['label']= df_train['감정_대분류'].map(emotion_map)
df_test['label']= df_test['감정_대분류'].map(emotion_map)

In [ ]:
#가사 라벨링용 입력데이터
na_emotion_map = {"불안":0,"상처":1,"슬픔":2,"당황":3,"기쁨":4}
nadf_train['label']= nadf_train['감정_대분류'].map(na_emotion_map)
nadf_test['label']= nadf_test['감정_대분류'].map(na_emotion_map)

In [ ]:
df_train= df_train.loc[:,['문장','label']]
df_test= df_test.loc[:,['문장','label']]
nadf_train= df_train.loc[:,['문장','label']]
nadf_test= df_test.loc[:,['문장','label']]
#df_test.tail(4)

In [ ]:
SEQ_LEN = 150 #문장의 최대 길이
BATCH_SIZE = 16 #배치 사이즈
EPOCHS=2
LR=1e-5

pretrained_path ="/your_path/BERT"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "문장"
LABEL_COLUMN = "label"

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

In [ ]:
#토크나이저
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [ ]:
tokenizer = inherit_Tokenizer(token_dict)

#데이터프레임 넣어서 그걸로 토큰화
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [ ]:
train_x, train_y = load_data(df_train)
test_x, test_y = load_data(df_test)

In [ ]:
#자꾸 Keyerror가 나서 보니까 분노 데이터를 제공하고 난 데이터의 index가 뒤죽박죽이었다.
nadf_train.reset_index(inplace=True)
nadf_test.reset_index(inplace=True)
del nadf_train['index']
del nadf_test['index']

In [ ]:
na_train_x, na_train_y = load_data(nadf_train)
na_test_x, na_test_y = load_data(nadf_test)

100%|██████████| 6641/6641 [00:00<00:00, 8520.12it/s]


In [ ]:
layer_num = 12
#모델 불러오기
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [ ]:
#문장 분류할 때 쓸 모델
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output


  outputs = keras.layers.Dense(units=6, activation='softmax',
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name="classifier")(dense)

  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss=losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])
  
  return bert_model

In [ ]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

Train on 51601 samples, validate on 6641 samples
Epoch 1/2
12016/51601 [=====>........................] - ETA: 24:13 - loss: nan - acc: 0.1798

KeyboardInterrupt: ignored

In [ ]:
#문장 분류할 때 쓸 모델
#bert_model.save_weights(/your_path/+"/bert.h5")
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights(path+"/bert.h5")

InvalidArgumentError: ignored

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x = predict_convert_data(data_df)

    return data_x

In [ ]:
test_set = predict_load_data(df_test)
#예측
preds = bert_model.predict(test_set)

100%|██████████| 5384/5384 [00:01<00:00, 3969.70it/s]
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
preds

array([[0.02534179, 0.03346704, 0.03166079, 0.9057006 , 0.00382969],
       [0.85384786, 0.03535932, 0.02508602, 0.08190849, 0.00379826],
       [0.6832798 , 0.03235516, 0.0169117 , 0.2629731 , 0.00448023],
       ...,
       [0.3138968 , 0.2613729 , 0.21032389, 0.21133478, 0.00307169],
       [0.1377151 , 0.30384177, 0.20128794, 0.34460136, 0.01255383],
       [0.27271774, 0.29068658, 0.18386415, 0.2429661 , 0.00976542]],
      dtype=float32)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix


preds = np.argmax(preds,axis=1) #그냥하니까안됨...
y_true = df_test['label']
# F1 Score 확인
print(classification_report(y_true, preds))

              precision    recall  f1-score   support

           0       0.57      0.79      0.66      1113
           1       0.66      0.49      0.56      1007
           2       0.62      0.66      0.64      1003
           3       0.70      0.59      0.64      1048
           4       0.92      0.86      0.89      1213

    accuracy                           0.69      5384
   macro avg       0.69      0.68      0.68      5384
weighted avg       0.70      0.69      0.69      5384



In [ ]:
#가사 라벨링
def lyrics_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def evaluation_predict(sentence):
    data_x = lyrics_convert_data(sentence)
    predict = bert_model.predict(data_x)
    #라벨링할 땐 분노 데이터를 뺀 것으로 따로 모델을 학습시켜서 사용했으나, 해당 코드에는 적용하지 않았습니다.
    #위의 get_bert_finetuning_model 부분을 복붙하여 따로 학습하세요!
    predict_answer = np.argmax(np.ravel(predict))
    #print(predict_answer)
    if predict_answer == 0:
      return "불안"
    elif predict_answer == 1:
      return "상처"
    elif predict_answer ==2:
      return "슬픔"
    elif predict_answer ==3:
      return "당황"
    elif predict_answer ==4:
      return "기쁨"

In [ ]:
#노래 리스트 불러오고 가공
song_list=pd.read_csv("/content/gdrive/MyDrive/MultiProject/music_genre_total.csv", encoding = "utf-8", header = 0, index_col = 0)
song_list = song_list.iloc[:,1:]
#중복 노래 제거
song_list = song_list.drop_duplicates('title')
song_list = song_list.reset_index(drop=True)
nolyric = song_list[song_list['lyric'].isnull()]
#nolyric.genre.unique()
#포크 블루스 이런거 소수면 살리려 했는데 그냥 누락되어 보이는 데이터도 많으므로 쿨하게 드랍함
song_list = song_list.dropna(axis='index',how='any')
song_list

,title,artist,lyric,genre
0,내 번호를 지워줘,박수진,이제는 사랑 같은 거 다시는 안 할 거라며 헤어지던 날 다짐했었는데 마음처럼 쉽지 ...,발라드
1,빛 바래 사라지지만,마린,언젠가 함께 살자고 그저 바라보며 살면 된다고 몸 뉘일 작은방 한 칸과 그대만 있으...,"발라드, 인디음악"
2,설월화 (Vocal 이서한),그네,사시사철 좋은 풍경을 바라보며 해를 보냈죠 많은 걸 놓고 적은 걸 얻은 그런 해를 ...,"발라드, 인디음악"
3,작은별,아웃오브캠퍼스,너는 참 특별한 작은 별 작지만 선명하게 꼭 필요한 곳을 비추지 너는 참 용감한 작...,"발라드, 인디음악"
4,미안해,SOI,미안해 내가 날 많이 아껴준 너인데 후회하겠지만 이제는 말할게 미안해 정말 미안해 ...,"발라드, 인디음악"
...,...,...,...,...
11591,네가 바로 지구야,이성구,네가 바로 지구야 지구는 바로 너야 네가 바로 희망이야 희망은 바로 너야 네가 보는...,"인디음악, 포크/블루스"
11593,동화.1,조경윤,커피를 너무 마셔서 밤을 새운 어떤 새벽에 좋은 향기가 가득한 내 슬픔은 강아지들의...,"인디음악, 포크/블루스"
11594,"오늘의 계절 (Feat. 진예주, 윤준태)",팔레트 (Palette),나무들이 하나 둘씩 물들 때 우리는 추억에 물들었는데 이제는 나무들이 물들어도 추억...,포크/블루스
11595,캠퍼스,오느린윤혜린,자 이제 시작해 새로운 하루 아름다운 하늘을 좀 봐 저 푸른 세상엔 뭐가 있을까 알...,"인디음악, 포크/블루스"


In [ ]:
song_list['emotion'] = song_list['lyric'].apply(evaluation_predict)
song_list

,title,artist,lyric,genre,emotion
0,내 번호를 지워줘,박수진,이제는 사랑 같은 거 다시는 안 할 거라며 헤어지던 날 다짐했었는데 마음처럼 쉽지 ...,발라드,슬픔
1,빛 바래 사라지지만,마린,언젠가 함께 살자고 그저 바라보며 살면 된다고 몸 뉘일 작은방 한 칸과 그대만 있으...,"발라드, 인디음악",슬픔
2,설월화 (Vocal 이서한),그네,사시사철 좋은 풍경을 바라보며 해를 보냈죠 많은 걸 놓고 적은 걸 얻은 그런 해를 ...,"발라드, 인디음악",불안
3,작은별,아웃오브캠퍼스,너는 참 특별한 작은 별 작지만 선명하게 꼭 필요한 곳을 비추지 너는 참 용감한 작...,"발라드, 인디음악",슬픔
4,미안해,SOI,미안해 내가 날 많이 아껴준 너인데 후회하겠지만 이제는 말할게 미안해 정말 미안해 ...,"발라드, 인디음악",슬픔
...,...,...,...,...,...
11591,네가 바로 지구야,이성구,네가 바로 지구야 지구는 바로 너야 네가 바로 희망이야 희망은 바로 너야 네가 보는...,"인디음악, 포크/블루스",상처
11593,동화.1,조경윤,커피를 너무 마셔서 밤을 새운 어떤 새벽에 좋은 향기가 가득한 내 슬픔은 강아지들의...,"인디음악, 포크/블루스",슬픔
11594,"오늘의 계절 (Feat. 진예주, 윤준태)",팔레트 (Palette),나무들이 하나 둘씩 물들 때 우리는 추억에 물들었는데 이제는 나무들이 물들어도 추억...,포크/블루스,슬픔
11595,캠퍼스,오느린윤혜린,자 이제 시작해 새로운 하루 아름다운 하늘을 좀 봐 저 푸른 세상엔 뭐가 있을까 알...,"인디음악, 포크/블루스",불안


In [ ]:
song_list.to_csv('/content/gdrive/MyDrive/MultiProject/music_total_labeled_0214.csv', index=False)

In [ ]:
#분류 잘 되어있는지 확인
song_list[song_list.emotion == '슬픔']

,title,artist,lyric,genre,emotion
0,내 번호를 지워줘,박수진,이제는 사랑 같은 거 다시는 안 할 거라며 헤어지던 날 다짐했었는데 마음처럼 쉽지 ...,발라드,슬픔
1,빛 바래 사라지지만,마린,언젠가 함께 살자고 그저 바라보며 살면 된다고 몸 뉘일 작은방 한 칸과 그대만 있으...,"발라드, 인디음악",슬픔
3,작은별,아웃오브캠퍼스,너는 참 특별한 작은 별 작지만 선명하게 꼭 필요한 곳을 비추지 너는 참 용감한 작...,"발라드, 인디음악",슬픔
4,미안해,SOI,미안해 내가 날 많이 아껴준 너인데 후회하겠지만 이제는 말할게 미안해 정말 미안해 ...,"발라드, 인디음악",슬픔
6,Delusion (Feat. Chordgee),Hasilver (하실버),꿈에 네가 나왔어 그 날 하루 웬종일 기분이 그냥 좋아 원래 널 좋아헀나 네가 나왔...,발라드,슬픔
...,...,...,...,...,...
11584,좋은데 슬퍼,낮은음자리,너 많이 변했다 안 그랬었는데 무뚝뚝한 말투가 많이 다정해졌네 웃기도 잘 웃고 집안...,포크/블루스,슬픔
11586,추억의 길 (Feat. 공명),주하주,하얀눈이 꽃잎처럼 흩날리는 이 길은 하얀교복 그 소녀와 설레이며 걷던 길 푸르르던 ...,포크/블루스,슬픔
11593,동화.1,조경윤,커피를 너무 마셔서 밤을 새운 어떤 새벽에 좋은 향기가 가득한 내 슬픔은 강아지들의...,"인디음악, 포크/블루스",슬픔
11594,"오늘의 계절 (Feat. 진예주, 윤준태)",팔레트 (Palette),나무들이 하나 둘씩 물들 때 우리는 추억에 물들었는데 이제는 나무들이 물들어도 추억...,포크/블루스,슬픔


In [ ]:
import pandas as pd
song_list_tot=pd.read_csv("/content/gdrive/MyDrive/MultiProject/music_total_labeled_0214.csv", encoding = "utf-8", header = 0, index_col = 0)
song_list_tot

,artist,lyric,genre,emotion
title,,,,
내 번호를 지워줘,박수진,이제는 사랑 같은 거 다시는 안 할 거라며 헤어지던 날 다짐했었는데 마음처럼 쉽지 ...,발라드,슬픔
빛 바래 사라지지만,마린,언젠가 함께 살자고 그저 바라보며 살면 된다고 몸 뉘일 작은방 한 칸과 그대만 있으...,"발라드, 인디음악",슬픔
설월화 (Vocal 이서한),그네,사시사철 좋은 풍경을 바라보며 해를 보냈죠 많은 걸 놓고 적은 걸 얻은 그런 해를 ...,"발라드, 인디음악",불안
작은별,아웃오브캠퍼스,너는 참 특별한 작은 별 작지만 선명하게 꼭 필요한 곳을 비추지 너는 참 용감한 작...,"발라드, 인디음악",슬픔
미안해,SOI,미안해 내가 날 많이 아껴준 너인데 후회하겠지만 이제는 말할게 미안해 정말 미안해 ...,"발라드, 인디음악",슬픔
...,...,...,...,...
네가 바로 지구야,이성구,네가 바로 지구야 지구는 바로 너야 네가 바로 희망이야 희망은 바로 너야 네가 보는...,"인디음악, 포크/블루스",상처
동화.1,조경윤,커피를 너무 마셔서 밤을 새운 어떤 새벽에 좋은 향기가 가득한 내 슬픔은 강아지들의...,"인디음악, 포크/블루스",슬픔
"오늘의 계절 (Feat. 진예주, 윤준태)",팔레트 (Palette),나무들이 하나 둘씩 물들 때 우리는 추억에 물들었는데 이제는 나무들이 물들어도 추억...,포크/블루스,슬픔


In [ ]:
#테스트: 문장 분류해서 결과 내기
def sentence_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def sentence_predict_save(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)
    predict_answer = np.argmax(np.ravel(predict))
    #print(predict_answer)
    if predict_answer == 0:
      return "불안"
    elif predict_answer == 1:
      return "분노"
    elif predict_answer ==2:
      return "상처"
    elif predict_answer ==3:
      return "슬픔"
    elif predict_answer ==4:
      return "당황"
    elif predict_answer ==5:
      return "기쁨"

In [ ]:
def howareyou(sentence):
    saved_user = pd.DataFrame({'문장': [sentence]})
    saved_user['emotion'] = saved_user['문장'].apply(sentence_predict_save)
    return saved_user

def music_recommend():
    sentence = input("오늘의 기분을 한줄로 입력")
    output_val = howareyou(sentence)
    emotion = output_val.iloc[0]['emotion']
    print(f"감정:{emotion}")
    if emotion == '불안':
        return song_list_tot[song_list_tot['emotion'] == '불안']
    elif emotion == '분노':
        return song_list_tot[song_list_tot['emotion'] == '기쁨']
    elif emotion == '상처':
        return song_list_tot[song_list_tot['emotion'] == '상처']
    elif emotion == '슬픔':
        return song_list_tot[song_list_tot['emotion'] == '슬픔']
    elif emotion == '당황':
        return song_list_tot[song_list_tot['emotion'] == '기쁨']
    elif emotion == '기쁨':
        return song_list_tot[song_list_tot['emotion'] == '기쁨']
    # 저 리턴된 saved_user 받아와서
    # song_list_tot에 맞게 매칭해서 출력(기준은 정해야됨)

In [ ]:
music_recommend()
#이제 스트림릿에 적용하기만 하면 됨!

오늘의 기분을 한줄로 입력면접 볼때는 야근이 없다고 했는데 지금 3일째 야근중이네... 뭐야?
감정:상처


,artist,lyric,genre,emotion
title,,,,
일상(日常),가은(GaEun),모든 게 무겁기만 하고 모두가 무서워 이렇게 모든 것을 버틸 수조차 없을 때 아무것...,발라드,상처
너의 나에게 (feat. 박상은),HAPP,빈 종이에 괜히 네 이름을 쓰면서 깊이 머물던 너를 꺼내보고 누군가 나에게 행복이 ...,"발라드, 인디음악",상처
B.O.X,베이(BAE),작은 상자 안에 채워지지 않네 작은 상자 안에 채워지지 않네 사랑은 갈수록 더 고민...,"발라드, 인디음악",상처
Lost the way,사비나앤드론즈,닿을 것 같아서 잡힐 것 같아서 깊은 선택 위로 엉켜버린 미로 돌아갈 수 없어 떠나...,"발라드, 국내드라마",상처
잘해보고 싶어요,리아킴,잘해보고 싶어요 내가 하고 싶은 일을 오늘도 어제처럼 나 이렇게 지내요 세월이 가기...,발라드,상처
...,...,...,...,...
어젠 꿈,권소정,오늘도 내 하루가 저물어 가네요 어젯밤 꾸던 꿈이 아직 생생한데요 나는 널 생각보다...,"인디음악, 포크/블루스",상처
어쩌면 너도,하이투힘 (Hi to him),아무렇지 않게 또 하루를 보내고 결국 사는 게 비슷하다 말을 하고 이렇게 어른이 되...,포크/블루스,상처
그런 말 하지마,양말기획,그런 말 하지마 설명도 하지마 듣고 싶다고 난 말한 적 없잖아 그런 말 하지마 궁금...,포크/블루스,상처
